In [ ]:
--- ZEMA API based DAM LMP --- (Source S3)
CREATE
OR
REPLACE
VDS "ep_dso_zema"."ne_price_actuals_vds" AS
SELECT distinct interval_ending_time_utc              as date_utc,
                COALESCE(CAST(price as double), NULL) as systemlambda,
                type,
                upper(venue)                          as venue,
                upper(market)                         as market,
                lower(product)                        as product,
                lower(settlement_point)               as settlement_point
FROM "zema"."ne_price_actuals"
WHERE "year" >= (year(current_date ()) - 4)
----------

--- we need to have the same for CAISO --- (Source S3)
--- ERCOT Nodal Weather Assumption Data by Zone : Hourly --- (Source S3)
CREATE
OR
REPLACE
VDS "ep_dso_zema"."ercot_nodal_wthr_asmptn_vds" AS
SELECT date_utc,
       MAX(case when zone = 'Coast' and row_num = 1 then temperature end)         as coast,
       MAX(case when zone = 'East' and row_num = 1 then temperature end)          as east,
       MAX(case when zone = 'North Central' and row_num = 1 then temperature end) as north_central,
       MAX(case when zone = 'Far West' and row_num = 1 then temperature end)      as far_west,
       MAX(case when zone = 'Southern' and row_num = 1 then temperature end)      as southern,
       MAX(case when zone = 'North' and row_num = 1 then temperature end)         as north,
       MAX(case when zone = 'South Central' and row_num = 1 then temperature end) as south_central,
       MAX(case when zone = 'West' and row_num = 1 then temperature end)          as west
FROM (SELECT date_utc,
          zone,
          COALESCE (CAST (temperature as double), NULL) as temperature,
          row_number() OVER (partition by date_utc, zone order by date_utc desc) as row_num
      FROM "zema"."ercot_nodal_wthr_asmptn"
      WHERE "year" >= (year (current_date ()) - 4))
GROUP BY date_utc
----------

--- we need to have the same for CAISO --- (Source S3)
--- WSI Weather Data : Hourly --- (Source S3)
CREATE
OR
REPLACE
VDS "ep_dso_zema"."wsi_hourly_ta_shell_vds" AS
SELECT wsi_hourly_trader_actual_dt_utc as date_utc,
       round(avg(cloud_cover), 2)      as cloud_cover,
       round(avg(dewpoint), 2)         as dewpoint,
       round(avg(heat_index), 2)       as heat_index,
       round(avg(precipitation), 2)    as precipitation,
       round(avg(temperature), 2)      as temperature,
       round(avg(wind_chill), 2)       as wind_chill,
       round(avg(wind_direction), 2)   as wind_direction,
       round(avg(wind_speed), 2)       as wind_speed
FROM "zema"."wsi_hourly_ta_shell"
WHERE "year" >= (year(current_date ()) - 4)
GROUP BY wsi_hourly_trader_actual_dt_utc
----------


--- Load / Consumption Data By Zone : Hourly --- (Source S3)
CREATE
OR
REPLACE
VDS "ep_dso_zema"."ne_caiso_demand_forecasts_vds" AS
SELECT date_utc,
       MAX(case when asset_code = 'LOAD_ZONE_SCE-TAC' and row_num = 1 then fc_load end)      as load_lz_sce_tac,
       MAX(case when asset_code = 'LOAD_ZONE_MWD-TAC' and row_num = 1 then fc_load end)       as load_lz_mwd_tac,
       MAX(case when asset_code = 'LOAD_ZONE_NEVP' and row_num = 1 then fc_load end)          as load_lz_nevp,
       MAX(case when asset_code = 'LOAD_ZONE_PGE' and row_num = 1 then fc_load end)          as load_lz_pge,
       MAX(case when asset_code = 'LOAD_ZONE_PACW' and row_num = 1 then fc_load end)         as load_lz_pacw,
       MAX(case when asset_code = 'LOAD_ZONE_IPCO' and row_num = 1 then fc_load end)         as load_lz_ipco,
       MAX(case when asset_code = 'LOAD_ZONE_PACE' and row_num = 1 then fc_load end)        as load_lz_pace,
       MAX(case when asset_code = 'LOAD_ZONE_SCL' and row_num = 1 then fc_load end)         as load_lz_scl,
       MAX(case when asset_code = 'LOAD_ZONE_VEA-TAC' and row_num = 1 then fc_load end)     as load_lz_vea_tac,
       MAX(case when asset_code = 'LOAD_ZONE_CA-ISO-TAC' and row_num = 1 then fc_load end)  as load_system_level,
       MAX(case when asset_code = 'LOAD_ZONE_SRP' and row_num = 1 then fc_load end)         as load_lz_srp,
       MAX(case when asset_code = 'LOAD_ZONE_PGE-TAC' and row_num = 1 then fc_load end)     as load_lz_pge_tac,
       MAX(case when asset_code = 'LOAD_ZONE_SDGE-TAC' and row_num = 1 then fc_load end)    as load_lz_sdge_tac,
       MAX(case when asset_code = 'LOAD_ZONE_PSEI' and row_num = 1 then fc_load end)        as load_lz_psei,
       MAX(case when asset_code = 'LOAD_ZONE_BANCSMUD' and row_num = 1 then fc_load end)    as load_lz_bancsmud,
       MAX(case when asset_code = 'LOAD_ZONE_AZPS' and row_num = 1 then fc_load end)        as load_lz_azps
       
FROM (SELECT interval_ending_time_utc                as date_utc,
             asset_code,
             COALESCE(CAST("value" as double), NULL) as fc_load,
             row_number()
                                                        OVER (partition by interval_ending_time_utc,asset_code order by source_created_at desc) as row_num
      FROM "zema"."ne_energy_forecasts"
      WHERE "year" >= (year(current_date ()) - 4)
        AND asset_code IS NOT NULL
        AND ((provider = 'ZEMA_MRTU_SLD_FCST_DAM_V'      and model_id = 'CAISO_LOAD_FRC_DAM' and type = 'C'))
     )
GROUP BY date_utc
----------


--- Generation Data By Zone : Hourly --- (Source S3)
CREATE
OR
REPLACE
VDS "ep_dso_zema"."ne_caiso_energy_forecasts_vds" AS
SELECT date_utc,
       
       MAX(case when asset_code = 'SOLAR_NP15' and row_num = 1 then fc_load end)            as solar_np15,
       MAX(case when asset_code = 'WIND_NP15' and row_num = 1 then fc_load end)             as wind_np15,
       MAX(case when asset_code = 'SOLAR_SP15' and row_num = 1 then fc_load end)            as solar_sp15,
       MAX(case when asset_code = 'WIND_SP15' and row_num = 1 then fc_load end)             as wind_sp15,
       MAX(case when asset_code = 'SOLAR_ZP26' and row_num = 1 then fc_load end)            as solar_zp26
       
FROM (SELECT interval_ending_time_utc                as date_utc,
             asset_code,
             COALESCE(CAST("value" as double), NULL) as fc_load,
             row_number()
                                                        OVER (partition by interval_ending_time_utc,asset_code order by source_created_at desc) as row_num
      FROM "zema"."ne_energy_forecasts"
      WHERE "year" >= (year(current_date ()) - 4)
        AND asset_code IS NOT NULL
        AND ((provider = 'ZEMA_MRTU_SLD_REN_FCST_DAM'    and model_id = 'CAISO_GEN_FRC_SOLAR_DAM' and type = 'G')
          OR (provider = 'ZEMA_MRTU_SLD_REN_FCST_DAM'    and model_id = 'CAISO_GEN_FRC_WIND_DAM' and type = 'G'))
     )
GROUP BY date_utc
----------


--- Load / Consumption Data By Zone : Real Time --- (Source S3)
CREATE
OR
REPLACE
VDS "ep_dso_zema"."ne_caiso_energy_forecasts_rt_vds" AS
SELECT date_utc,
       MAX(case when asset_code = 'LOAD_ZONE_SCE-TAC' and row_num = 1 then fc_load end)      as load_lz_sce_tac_rt,
       MAX(case when asset_code = 'LOAD_ZONE_MWD-TAC' and row_num = 1 then fc_load end)       as load_lz_mwd_tac_rt,
       MAX(case when asset_code = 'LOAD_ZONE_NEVP' and row_num = 1 then fc_load end)          as load_lz_nevp_rt,
       MAX(case when asset_code = 'LOAD_ZONE_PGE' and row_num = 1 then fc_load end)          as load_lz_pge_rt,
       MAX(case when asset_code = 'LOAD_ZONE_PACW' and row_num = 1 then fc_load end)         as load_lz_pacw_rt,
       MAX(case when asset_code = 'LOAD_ZONE_IPCO' and row_num = 1 then fc_load end)         as load_lz_ipco_rt,
       MAX(case when asset_code = 'LOAD_ZONE_PACE' and row_num = 1 then fc_load end)        as load_lz_pace_rt,
       MAX(case when asset_code = 'LOAD_ZONE_SCL' and row_num = 1 then fc_load end)         as load_lz_scl_rt,
       MAX(case when asset_code = 'LOAD_ZONE_VEA-TAC' and row_num = 1 then fc_load end)     as load_lz_vea_tac_rt,
       MAX(case when asset_code = 'LOAD_ZONE_CA-ISO-TAC' and row_num = 1 then fc_load end)  as load_system_level_rt,
       MAX(case when asset_code = 'LOAD_ZONE_SRP' and row_num = 1 then fc_load end)         as load_lz_srp_rt,
       MAX(case when asset_code = 'LOAD_ZONE_PGE-TAC' and row_num = 1 then fc_load end)     as load_lz_pge_tac_rt,
       MAX(case when asset_code = 'LOAD_ZONE_SDGE-TAC' and row_num = 1 then fc_load end)    as load_lz_sdge_tac_rt,
       MAX(case when asset_code = 'LOAD_ZONE_PSEI' and row_num = 1 then fc_load end)        as load_lz_psei_rt,
       MAX(case when asset_code = 'LOAD_ZONE_BANCSMUD' and row_num = 1 then fc_load end)    as load_lz_bancsmud_rt,
       MAX(case when asset_code = 'LOAD_ZONE_AZPS' and row_num = 1 then fc_load end)        as load_lz_azps_rt,
       MAX(case when asset_code = 'SOLAR_NP15' and row_num = 1 then fc_load end)            as solar_np15_hasp,
       MAX(case when asset_code = 'WIND_NP15' and row_num = 1 then fc_load end)             as wind_np15_hasp,
       MAX(case when asset_code = 'SOLAR_SP15' and row_num = 1 then fc_load end)            as solar_sp15_hasp,
       MAX(case when asset_code = 'WIND_SP15' and row_num = 1 then fc_load end)             as wind_sp15_hasp,
       MAX(case when asset_code = 'SOLAR_ZP26' and row_num = 1 then fc_load end)            as solar_zp26_hasp,
       MAX(case when asset_code = 'WIND_ZP26' and row_num = 1 then fc_load end)             as wind_zp26_hasp
FROM (
         SELECT interval_ending_time_utc                as date_utc,
                asset_code,
                COALESCE(CAST("value" as double), NULL) as fc_load,
                row_number()
                                                           OVER (partition by interval_ending_time_utc,asset_code order by source_created_at desc) as row_num
         FROM "zema"."ne_energy_forecasts"
         WHERE "year" >= (year(current_date ()) - 4)
           AND asset_code IS NOT NULL
          AND ((provider = 'ZEMA_MRTU_SLD_FCST_RTM_V'      and model_id = 'CAISO_LOAD_FRC_RTM' and type = 'C')
          OR (provider = 'ZEMA_MRTU_SLD_REN_FCST_HASP'    and model_id = 'CAISO_GEN_FRC_SOLAR_HASP' and type = 'G')
          OR (provider = 'ZEMA_MRTU_SLD_REN_FCST_HASP'    and model_id = 'CAISO_GEN_FRC_WIND_HASP' and type = 'G'))
     )
GROUP BY date_utc
----------

--- Ancillary Prices By Asset : Hourly --- (Source S3)
CREATE
OR
REPLACE
VDS "ep_dso_zema"."ne_caiso_asp_forecasts_vds" AS
SELECT date_utc,
       MAX(case when asset_code = 'AS_CAISO_NS_REQ_MAX_MW' and row_num = 1 then price end) as caiso_nspin_max,
       MAX(case when asset_code = 'AS_CAISO_NS_REQ_MIN_MW' and row_num = 1 then price end) as caiso_nspin_min,
       MAX(case when asset_code = 'AS_CAISO_RD_REQ_MAX_MW' and row_num = 1 then price end) as caiso_regdn_max,
       MAX(case when asset_code = 'AS_CAISO_RD_REQ_MIN_MW' and row_num = 1 then price end) as caiso_regdn_min,
       MAX(case when asset_code = 'AS_CAISO_RU_REQ_MAX_MW' and row_num = 1 then price end) as caiso_regup_max,
       MAX(case when asset_code = 'AS_CAISO_RU_REQ_MIN_MW' and row_num = 1 then price end) as caiso_regup_min,
       MAX(case when asset_code = 'AS_CAISO_SP_REQ_MAX_MW' and row_num = 1 then price end) as caiso_spin_max,
       MAX(case when asset_code = 'AS_CAISO_SP_REQ_MIN_MW' and row_num = 1 then price end) as caiso_spin_min,
       MAX(case when asset_code = 'AS_CAISO_EXP_NS_REQ_MAX_MW' and row_num = 1 then price end) as caiso_exp_nspin_max,
       MAX(case when asset_code = 'AS_CAISO_EXP_NS_REQ_MIN_MW' and row_num = 1 then price end) as caiso_exp_nspin_min,
       MAX(case when asset_code = 'AS_CAISO_EXP_RD_REQ_MAX_MW' and row_num = 1 then price end) as caiso_exp_regdn_max,
       MAX(case when asset_code = 'AS_CAISO_EXP_RD_REQ_MIN_MW' and row_num = 1 then price end) as caiso_exp_regdn_min,
       MAX(case when asset_code = 'AS_CAISO_EXP_RU_REQ_MAX_MW' and row_num = 1 then price end) as caiso_exp_regup_max,
       MAX(case when asset_code = 'AS_CAISO_EXP_RU_REQ_MIN_MW' and row_num = 1 then price end) as caiso_exp_regup_min,
       MAX(case when asset_code = 'AS_CAISO_EXP_SP_REQ_MAX_MW' and row_num = 1 then price end) as caiso_exp_spin_max,
       MAX(case when asset_code = 'AS_CAISO_EXP_SP_REQ_MIN_MW' and row_num = 1 then price end) as caiso_exp_spin_min,
       MAX(case when asset_code = 'AS_CAISO_EXP_RMD_REQ_MAX_MW' and row_num = 1 then price end) as caiso_exp_rmd_max,
       MAX(case when asset_code = 'AS_CAISO_EXP_RMD_REQ_MIN_MW' and row_num = 1 then price end) as caiso_exp_rmd_min,
       MAX(case when asset_code = 'AS_CAISO_EXP_RMU_REQ_MAX_MW' and row_num = 1 then price end) as caiso_exp_spin_max,
       MAX(case when asset_code = 'AS_CAISO_EXP_RMU_REQ_MIN_MW' and row_num = 1 then price end) as caiso_exp_spin_min,
       MAX(case when asset_code = 'AS_NP26_NS_REQ_MAX_MW' and row_num = 1 then price end) as np26_nspin_max,
       MAX(case when asset_code = 'AS_NP26_NS_REQ_MIN_MW' and row_num = 1 then price end) as np26_nspin_min,
       MAX(case when asset_code = 'AS_NP26_RD_REQ_MAX_MW' and row_num = 1 then price end) as np26_regdn_max,
       MAX(case when asset_code = 'AS_NP26_RD_REQ_MIN_MW' and row_num = 1 then price end) as np26_regdn_min,
       MAX(case when asset_code = 'AS_NP26_RU_REQ_MAX_MW' and row_num = 1 then price end) as np26_regup_max,
       MAX(case when asset_code = 'AS_NP26_RU_REQ_MIN_MW' and row_num = 1 then price end) as np26_regup_min,
       MAX(case when asset_code = 'AS_NP26_SP_REQ_MAX_MW' and row_num = 1 then price end) as np26_spin_max,
       MAX(case when asset_code = 'AS_NP26_SP_REQ_MIN_MW' and row_num = 1 then price end) as np26_spin_min,
       MAX(case when asset_code = 'AS_NP26_EXP_NS_REQ_MAX_MW' and row_num = 1 then price end) as np26_exp_nspin_max,
       MAX(case when asset_code = 'AS_NP26_EXP_NS_REQ_MIN_MW' and row_num = 1 then price end) as np26_exp_nspin_min,
       MAX(case when asset_code = 'AS_NP26_EXP_RD_REQ_MAX_MW' and row_num = 1 then price end) as np26_exp_regdn_max,
       MAX(case when asset_code = 'AS_NP26_EXP_RD_REQ_MIN_MW' and row_num = 1 then price end) as np26_exp_regdn_min,
       MAX(case when asset_code = 'AS_NP26_EXP_RU_REQ_MAX_MW' and row_num = 1 then price end) as np26_exp_regup_max,
       MAX(case when asset_code = 'AS_NP26_EXP_RU_REQ_MIN_MW' and row_num = 1 then price end) as np26_exp_regup_min,
       MAX(case when asset_code = 'AS_NP26_EXP_SP_REQ_MAX_MW' and row_num = 1 then price end) as np26_exp_spin_max,
       MAX(case when asset_code = 'AS_NP26_EXP_SP_REQ_MIN_MW' and row_num = 1 then price end) as np26_exp_spin_min,
       MAX(case when asset_code = 'AS_SP26_NS_REQ_MAX_MW' and row_num = 1 then price end) as sp26_nspin_max,
       MAX(case when asset_code = 'AS_SP26_NS_REQ_MIN_MW' and row_num = 1 then price end) as sp26_nspin_min,
       MAX(case when asset_code = 'AS_SP26_RD_REQ_MAX_MW' and row_num = 1 then price end) as sp26_regdn_max,
       MAX(case when asset_code = 'AS_SP26_RD_REQ_MIN_MW' and row_num = 1 then price end) as sp26_regdn_min,
       MAX(case when asset_code = 'AS_SP26_RU_REQ_MAX_MW' and row_num = 1 then price end) as sp26_regup_max,
       MAX(case when asset_code = 'AS_SP26_RU_REQ_MIN_MW' and row_num = 1 then price end) as sp26_regup_min,
       MAX(case when asset_code = 'AS_SP26_SP_REQ_MAX_MW' and row_num = 1 then price end) as sp26_spin_max,
       MAX(case when asset_code = 'AS_SP26_SP_REQ_MIN_MW' and row_num = 1 then price end) as sp26_spin_min,
       MAX(case when asset_code = 'AS_SP26_EXP_NS_REQ_MAX_MW' and row_num = 1 then price end) as sp26_exp_nspin_max,
       MAX(case when asset_code = 'AS_SP26_EXP_NS_REQ_MIN_MW' and row_num = 1 then price end) as sp26_exp_nspin_min,
       MAX(case when asset_code = 'AS_SP26_EXP_RD_REQ_MAX_MW' and row_num = 1 then price end) as sp26_exp_regdn_max,
       MAX(case when asset_code = 'AS_SP26_EXP_RD_REQ_MIN_MW' and row_num = 1 then price end) as sp26_exp_regdn_min,
       MAX(case when asset_code = 'AS_SP26_EXP_RU_REQ_MAX_MW' and row_num = 1 then price end) as sp26_exp_regup_max,
       MAX(case when asset_code = 'AS_SP26_EXP_RU_REQ_MIN_MW' and row_num = 1 then price end) as sp26_exp_regup_min,
       MAX(case when asset_code = 'AS_SP26_EXP_SP_REQ_MAX_MW' and row_num = 1 then price end) as sp26_exp_spin_max,
       MAX(case when asset_code = 'AS_SP26_EXP_SP_REQ_MIN_MW' and row_num = 1 then price end) as sp26_exp_spin_min
       
FROM (
         SELECT interval_ending_time_utc                as date_utc,
                asset_code,
                COALESCE(CAST("value" as double), NULL) as price,
                row_number()
                                                           OVER (partition by interval_ending_time_utc,asset_code order by source_created_at desc) as row_num
         FROM "zema"."ne_energy_forecasts"
         WHERE "year" >= (year(current_date ()) - 4)
           AND asset_code IS NOT NULL
           AND (provider = 'ZEMA_MRTU_AS_REQ_DAM_V' and model_id = 'CAISO_AS_REQ_DAM' and type = 'AS_REQUIREMENT')
     )
GROUP BY date_utc
----------

----------

--- Forecast Outages --- (Source S3)
CREATE
OR
REPLACE
VDS "ep_dso_zema"."ne_caiso_energy_outage_forecasts_vds" AS
SELECT date_utc,
       MAX(case when asset_code = 'Hydro_NP15' and row_num = 1 then outage_quantity end) as np15_hydro_gen_out,
       MAX(case when asset_code = 'Thermal_NP15' and row_num = 1 then outage_quantity end) as np15_thermal_gen_out,
       MAX(case when asset_code = 'Renewable_NP15' and row_num = 1 then outage_quantity end) as np15_ren_gen_out,
       MAX(case when asset_code = 'Aggregated_NP15' and row_num = 1 then outage_quantity end) as np15_agg_gen_out,
       MAX(case when asset_code = 'Hydro_SP15' and row_num = 1 then outage_quantity end) as sp15_hydro_gen_out,
       MAX(case when asset_code = 'Thermal_SP15' and row_num = 1 then outage_quantity end) as sp15_thermal_gen_out,
       MAX(case when asset_code = 'Renewable_SP15' and row_num = 1 then outage_quantity end) as sp15_ren_gen_out,
       MAX(case when asset_code = 'Aggregated_SP15' and row_num = 1 then outage_quantity end) as sp15_agg_gen_out,
       MAX(case when asset_code = 'Hydro_ZP26' and row_num = 1 then outage_quantity end) as zp26_hydro_gen_out,
       MAX(case when asset_code = 'Thermal_ZP26' and row_num = 1 then outage_quantity end) as zp26_thermal_gen_out,
       MAX(case when asset_code = 'Renewable_ZP26' and row_num = 1 then outage_quantity end) as zp26_ren_gen_out,
       MAX(case when asset_code = 'Aggregated_ZP26' and row_num = 1 then outage_quantity end) as zp26_agg_gen_out
FROM (
         SELECT interval_ending_time_utc                        as date_utc,
                asset_code,
                COALESCE(CAST(outage_quantity as double), NULL) as outage_quantity,
                row_number()
                                                                   OVER (partition by interval_ending_time_utc,asset_code order by source_created_at desc) as row_num
         FROM "zema"."ne_energy_outage_forecasts"
         WHERE "year" >= (year(current_date ()) - 4)
           AND outage_quantity IS NOT NULL
           AND asset_code IS NOT NULL
           AND provider = 'ZEMA_MRTU_AGGR_OUTAGE_SCH'
     )
GROUP BY date_utc
----------

---we need to have it for CAISO----
--- Gas Prices --- (Source S3)
CREATE
OR
REPLACE
VDS "ep_dso_zema"."ngi_daily_vds" AS
SELECT distinct gas_price_dt_utc                  as date_utc,
                cast(gas_price_average as double) as gas_price_average,
                cast(deals as double)             as deals,
                cast(high as double)              as high,
                cast(low as double)               as low,
                cast(volume as double)            as volume
FROM "zema"."ngi_daily"
WHERE "year" >= (year(current_date ()) - 4)
  AND gas_price_average IS NOT NULL
----------

--- CAISO System Load & Resource Schedules: Hourly --- (Source S3)
CREATE
OR REPLACE VDS "ep_dso_zema"."ne_caiso_slrs_vds" AS
SELECT date_utc,
       MAX(case when asset_code = 'GEN_TAC_NCNTR' and row_num = 1 then price end)     as gen_ncntr,
       MAX(case when asset_code = 'GEN_Caiso_Totals' and row_num = 1 then price end)  as gen_total,
       MAX(case when asset_code = 'GEN_TAC_ECNTR' and row_num = 1 then price end)     as gen_ecntr,
       MAX(case when asset_code = 'GEN_TAC_NORTH' and row_num = 1 then price end)     as gen_north,
       MAX(case when asset_code = 'GEN_TAC_SOUTH' and row_num = 1 then price end)     as gen_south, 
       MAX(case when asset_code = 'LOAD_TAC_NCNTR' and row_num = 1 then price end)     as load_ncntr,
       MAX(case when asset_code = 'LOAD_Caiso_Totals' and row_num = 1 then price end)  as load_total,
       MAX(case when asset_code = 'LOAD_TAC_ECNTR' and row_num = 1 then price end)     as load_ecntr,
       MAX(case when asset_code = 'LOAD_TAC_NORTH' and row_num = 1 then price end)     as load_north,
       MAX(case when asset_code = 'LOAD_TAC_SOUTH' and row_num = 1 then price end)     as load_south, 
       MAX(case when asset_code = 'LOAD_NONTAC' and row_num = 1 then price end)        as load_nontac,
       MAX(case when asset_code = 'EXP_Caiso_Totals' and row_num = 1 then price end)  as exp_total,
       MAX(case when asset_code = 'EXP_TAC_ECNTR' and row_num = 1 then price end)     as exp_ecntr,
       MAX(case when asset_code = 'EXP_TAC_NORTH' and row_num = 1 then price end)     as exp_north,
       MAX(case when asset_code = 'EXP_TAC_SOUTH' and row_num = 1 then price end)     as exp_south, 
       MAX(case when asset_code = 'IMP_Caiso_Totals' and row_num = 1 then price end)  as imp_total,
       MAX(case when asset_code = 'IMP_TAC_ECNTR' and row_num = 1 then price end)     as imp_ecntr,
       MAX(case when asset_code = 'IMP_TAC_NORTH' and row_num = 1 then price end)     as imp_north,
       MAX(case when asset_code = 'IMP_TAC_SOUTH' and row_num = 1 then price end)     as imp_south,
       MAX(case when asset_code = 'IMP_TAC_NCNTR' and row_num = 1 then price end)     as exp_ncntr
     

FROM (
         SELECT interval_ending_time_utc                as date_utc,
                asset_code,
                COALESCE(CAST("value" as double), NULL) as price,
                row_number()                               OVER (partition by interval_ending_time_utc,asset_code order by source_created_at desc) as row_num
         FROM "zema"."ne_energy_forecasts"
         WHERE "year" >= (year(current_date ()) - 4)
           AND asset_code IS NOT NULL
           AND provider in ('ZEMA_MRTU_ENE_SLRS_DAM_V')
     )
GROUP BY date_utc
----------

--- CAISO System Load & Resource Schedules: Real-Time --- (Source S3)
    
CREATE
OR REPLACE VDS "ep_dso_zema"."ne_caiso_slrs_rt_vds" AS
SELECT date_utc,
       MAX(case when asset_code = 'GEN_TAC_NCNTR' and row_num = 1 then price end)     as gen_ncntr,
       MAX(case when asset_code = 'GEN_Caiso_Totals' and row_num = 1 then price end)  as gen_total,
       MAX(case when asset_code = 'GEN_TAC_ECNTR' and row_num = 1 then price end)     as gen_ecntr,
       MAX(case when asset_code = 'GEN_TAC_NORTH' and row_num = 1 then price end)     as gen_north,
       MAX(case when asset_code = 'GEN_TAC_SOUTH' and row_num = 1 then price end)     as gen_south, 
       MAX(case when asset_code = 'GEN_TAC_AZPS' and row_num = 1 then price end)     as gen_azps,
       MAX(case when asset_code = 'GEN_TAC_BANCSMUD' and row_num = 1 then price end)  as gen_bancsmud,
       MAX(case when asset_code = 'GEN_TAC_BCHA' and row_num = 1 then price end)     as gen_bcha,
       MAX(case when asset_code = 'GEN_TAC_IPCO' and row_num = 1 then price end)     as gen_ipco,
       MAX(case when asset_code = 'GEN_TAC_NEVP' and row_num = 1 then price end)     as gen_nevp, 
       MAX(case when asset_code = 'GEN_TAC_PAC' and row_num = 1 then price end)     as gen_pac,
       MAX(case when asset_code = 'GEN_TAC_PGE' and row_num = 1 then price end)     as gen_pge,
       MAX(case when asset_code = 'GEN_TAC_PSEI' and row_num = 1 then price end)     as gen_psei,
       MAX(case when asset_code = 'GEN_TAC_SCL' and row_num = 1 then price end)     as gen_scl, 
       MAX(case when asset_code = 'GEN_TAC_SRP' and row_num = 1 then price end)     as gen_srp,
       MAX(case when asset_code = 'EXP_Caiso_Totals' and row_num = 1 then price end)  as exp_total,
       MAX(case when asset_code = 'EXP_TAC_ECNTR' and row_num = 1 then price end)     as exp_ecntr,
       MAX(case when asset_code = 'EXP_TAC_NORTH' and row_num = 1 then price end)     as exp_north,
       MAX(case when asset_code = 'EXP_TAC_SOUTH' and row_num = 1 then price end)     as exp_south, 
       MAX(case when asset_code = 'EXP_TAC_AZPS' and row_num = 1 then price end)     as exp_azps,
       MAX(case when asset_code = 'EXP_TAC_BANCSMUD' and row_num = 1 then price end)     as exp_bancsmud,
       MAX(case when asset_code = 'EXP_TAC_BCHA' and row_num = 1 then price end)     as exp_bcha, 
       MAX(case when asset_code = 'EXP_TAC_IPCO' and row_num = 1 then price end)     as exp_ipco,
       MAX(case when asset_code = 'EXP_TAC_NEVP' and row_num = 1 then price end)     as exp_nevp,
       MAX(case when asset_code = 'EXP_TAC_PAC' and row_num = 1 then price end)     as exp_pac, 
       MAX(case when asset_code = 'EXP_TAC_PGE' and row_num = 1 then price end)     as exp_pge,
       MAX(case when asset_code = 'EXP_TAC_PSEI' and row_num = 1 then price end)     as exp_psei, 
       MAX(case when asset_code = 'EXP_TAC_SCL' and row_num = 1 then price end)     as caiso_exp_scl,
       MAX(case when asset_code = 'EXP_TAC_SRP' and row_num = 1 then price end)     as caiso_exp_srp,   
       MAX(case when asset_code = 'IMP_Caiso_Totals' and row_num = 1 then price end)  as imp_total,
       MAX(case when asset_code = 'IMP_TAC_ECNTR' and row_num = 1 then price end)     as imp_ecntr,
       MAX(case when asset_code = 'IMP_TAC_NORTH' and row_num = 1 then price end)     as imp_north,
       MAX(case when asset_code = 'IMP_TAC_SOUTH' and row_num = 1 then price end)     as imp_south,
       MAX(case when asset_code = 'IMP_TAC_NCNTR' and row_num = 1 then price end)     as imp_ncntr,
       MAX(case when asset_code = 'IMP_TAC_AZPS' and row_num = 1 then price end)     as imp_azps,
       MAX(case when asset_code = 'IMP_TAC_BANCSMUD' and row_num = 1 then price end)     as imp_bancsmud,
       MAX(case when asset_code = 'IMP_TAC_BCHA' and row_num = 1 then price end)     as imp_bcha, 
       MAX(case when asset_code = 'IMP_TAC_IPCO' and row_num = 1 then price end)     as imp_ipco,
       MAX(case when asset_code = 'IMP_TAC_NEVP' and row_num = 1 then price end)     as imp_nevp,
       MAX(case when asset_code = 'IMP_TAC_PAC' and row_num = 1 then price end)     as imp_pac, 
       MAX(case when asset_code = 'IMP_TAC_PGE' and row_num = 1 then price end)     as imp_pge,
       MAX(case when asset_code = 'IMP_TAC_PSEI' and row_num = 1 then price end)     as imp_psei, 
       MAX(case when asset_code = 'IMP_TAC_SCL' and row_num = 1 then price end)     as imp_scl,
       MAX(case when asset_code = 'IMP_TAC_SRP' and row_num = 1 then price end)     as imp_srp   
     

FROM (
         SELECT interval_ending_time_utc                as date_utc,
                asset_code,
                COALESCE(CAST("value" as double), NULL) as price,
                row_number()                               OVER (partition by interval_ending_time_utc,asset_code order by source_created_at desc) as row_num
         FROM "zema"."ne_energy_forecasts"
         WHERE "year" >= (year(current_date ()) - 4)
           AND asset_code IS NOT NULL
           AND provider in ('ZEMA_MRTU_ENE_SLRS_RTM_V')
     )
GROUP BY date_utc
----------
